In [23]:
import os
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

import dotenv
dotenv.load_dotenv()

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [24]:
from absl import logging
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def get_embedding(text):
  return model(text)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [25]:
def get_distance(text1, text2):
   text1 = str(text1).lower()
   text2 = str(text2).lower()

   text1 = [text1]
   text2 = [text2]

   embedding1 = get_embedding(text1)
   embedding2 = get_embedding(text2)

   return float(np.inner(embedding1, embedding2)[0][0])

In [26]:
# go through sample.json and ask questions to gpt
import json
all_qs = json.load(open("filtered_train.json"))

# randomly sample 0 questions
import random
sample = random.sample(all_qs, 50)

In [27]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o-mini",
    # temperature=0.2,
    

)

str(chat_completion.choices[0].message.content)


'Hello! How can I assist you today?'

In [28]:
output = []

sum_1 = 0
sum_2 = 0

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning. The Question: {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o-mini"
    )


    ambig_answer_distance = 0
    disambig_answer_distance = 0
    for ans in i["nq_answer"]:
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        curr_disambig_answer_distance = get_distance(ans, final_answer.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance
        if curr_disambig_answer_distance > disambig_answer_distance:
            disambig_answer_distance = curr_disambig_answer_distance

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
        "question_distance": get_distance(i["nq_question"], chat_completion.choices[0].message.content),
        "answer_distance": get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content),
        "ambig_answer_distance": ambig_answer_distance,
        "disambig_answer_distance": disambig_answer_distance
    }

    sum_1 += curr["ambig_answer_distance"]
    sum_2 += curr["disambig_answer_distance"]

    # print(f"question_distance: {get_distance(i["nq_question"], chat_completion.choices[0].message.content)}")
    # print(f"answer_distance: {get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content)}")
    # if i["nq_answer"] is list:
    # print(f"ambig_answer_distance: {get_distance(i["nq_answer"], ootb_llm_rseponse.choices[0].message.content)}")
    # print(f"disambig_answer_distance: {get_distance(i["nq_answer"], final_answer.choices[0].message.content)}")

    output.append(curr)

    # save output to json file
    json.dump(output, open("disambig_prompt_4o_mini.json", "w"))

    print(curr)

print("="*20)
print(f"Average ambig answer distance: {sum_1/len(sample)}")
print(f"Average disambig answer distance: {sum_2/len(sample)}")


What is the name of the chief justice of the Gujarat High Court?
{'data_id': '7649235002594425144', 'ambig_question': 'Who is chief justice of gujarat high court?', 'ambig_prompt_response': 'Justice Aravind Kumar', 'disambig_question': 'What is the name of the chief justice of the Gujarat High Court?', 'disambig_prompt_response': 'Justice Sunita Agarwal.', 'ground_truth': ['Justice R. Subhash Reddy'], 'question_distance': 0.9179774522781372, 'answer_distance': 0.7007501125335693, 'ambig_answer_distance': 0.7283543348312378, 'disambig_answer_distance': 0.7812121510505676}
What U.S. president established the precedent of limiting presidential terms to two?
{'data_id': '-7521816944277648300', 'ambig_question': 'What u.s. president established the precedent of not seeking more than two terms in office?', 'ambig_prompt_response': 'George Washington', 'disambig_question': 'What U.S. president established the precedent of limiting presidential terms to two?', 'disambig_prompt_response': 'Fran

In [31]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o-mini",
    temperature=1.8,
    

)

str(chat_completion.choices[0].message.content)


'Hello! How can I assist you today?'

In [32]:
output = []

sum_1 = 0
sum_2 = 0

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning. The Question: {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o-mini"
    )


    ambig_answer_distance = 0
    disambig_answer_distance = 0
    for ans in i["nq_answer"]:
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        curr_disambig_answer_distance = get_distance(ans, final_answer.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance
        if curr_disambig_answer_distance > disambig_answer_distance:
            disambig_answer_distance = curr_disambig_answer_distance

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
        "question_distance": get_distance(i["nq_question"], chat_completion.choices[0].message.content),
        "answer_distance": get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content),
        "ambig_answer_distance": ambig_answer_distance,
        "disambig_answer_distance": disambig_answer_distance
    }

    sum_1 += curr["ambig_answer_distance"]
    sum_2 += curr["disambig_answer_distance"]

    # print(f"question_distance: {get_distance(i["nq_question"], chat_completion.choices[0].message.content)}")
    # print(f"answer_distance: {get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content)}")
    # if i["nq_answer"] is list:
    # print(f"ambig_answer_distance: {get_distance(i["nq_answer"], ootb_llm_rseponse.choices[0].message.content)}")
    # print(f"disambig_answer_distance: {get_distance(i["nq_answer"], final_answer.choices[0].message.content)}")

    output.append(curr)

    # save output to json file
    json.dump(output, open("disambig_prompt_4o_mini.json", "w"))

    print(curr)

print("="*20)
print(f"Average ambig answer distance: {sum_1/len(sample)}")
print(f"Average disambig answer distance: {sum_2/len(sample)}")


What is the name of the chief justice of the Gujarat High Court?
{'data_id': '7649235002594425144', 'ambig_question': 'Who is chief justice of gujarat high court?', 'ambig_prompt_response': 'Justice Ashish J. Desai', 'disambig_question': 'What is the name of the chief justice of the Gujarat High Court?', 'disambig_prompt_response': 'Justice Ashish S. Desai.', 'ground_truth': ['Justice R. Subhash Reddy'], 'question_distance': 0.9179774522781372, 'answer_distance': 0.9259400367736816, 'ambig_answer_distance': 0.7809098362922668, 'disambig_answer_distance': 0.8110467195510864}
What U.S. president established the precedent of limiting to two terms in office?
{'data_id': '-7521816944277648300', 'ambig_question': 'What u.s. president established the precedent of not seeking more than two terms in office?', 'ambig_prompt_response': 'George Washington', 'disambig_question': 'What U.S. president established the precedent of limiting to two terms in office?', 'disambig_prompt_response': 'Frankli

In [36]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o-mini",
    temperature=-0,
    

)

str(chat_completion.choices[0].message.content)


'Hello! How can I assist you today?'

In [37]:
output = []

sum_1 = 0
sum_2 = 0

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning. The Question: {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o-mini"
    )


    ambig_answer_distance = 0
    disambig_answer_distance = 0
    for ans in i["nq_answer"]:
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        curr_disambig_answer_distance = get_distance(ans, final_answer.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance
        if curr_disambig_answer_distance > disambig_answer_distance:
            disambig_answer_distance = curr_disambig_answer_distance

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
        "question_distance": get_distance(i["nq_question"], chat_completion.choices[0].message.content),
        "answer_distance": get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content),
        "ambig_answer_distance": ambig_answer_distance,
        "disambig_answer_distance": disambig_answer_distance
    }

    sum_1 += curr["ambig_answer_distance"]
    sum_2 += curr["disambig_answer_distance"]

    # print(f"question_distance: {get_distance(i["nq_question"], chat_completion.choices[0].message.content)}")
    # print(f"answer_distance: {get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content)}")
    # if i["nq_answer"] is list:
    # print(f"ambig_answer_distance: {get_distance(i["nq_answer"], ootb_llm_rseponse.choices[0].message.content)}")
    # print(f"disambig_answer_distance: {get_distance(i["nq_answer"], final_answer.choices[0].message.content)}")

    output.append(curr)

    # save output to json file
    json.dump(output, open("disambig_prompt_4o_mini.json", "w"))

    print(curr)

print("="*20)
print(f"Average ambig answer distance: {sum_1/len(sample)}")
print(f"Average disambig answer distance: {sum_2/len(sample)}")


What is the name of the chief justice of Gujarat High Court?
{'data_id': '7649235002594425144', 'ambig_question': 'Who is chief justice of gujarat high court?', 'ambig_prompt_response': 'Justice Ashish J. Desai', 'disambig_question': 'What is the name of the chief justice of Gujarat High Court?', 'disambig_prompt_response': 'Justice Aravind Kumar.', 'ground_truth': ['Justice R. Subhash Reddy'], 'question_distance': 0.9404118061065674, 'answer_distance': 0.7213051319122314, 'ambig_answer_distance': 0.7809098362922668, 'disambig_answer_distance': 0.7283543348312378}
What U.S. president established the precedent of two terms in office?
{'data_id': '-7521816944277648300', 'ambig_question': 'What u.s. president established the precedent of not seeking more than two terms in office?', 'ambig_prompt_response': 'George Washington', 'disambig_question': 'What U.S. president established the precedent of two terms in office?', 'disambig_prompt_response': 'George Washington', 'ground_truth': ['Geo